# Benin Solar Data EDA
Exploratory Data Analysis for the Benin solar dataset.

In [12]:
from charset_normalizer import detect
import pandas as pd
from scipy.stats import zscore

In [ ]:



with open('../data/solar-measurements_benin-malanville_qc.csv', 'rb') as file:
    result = detect(file.read())
    print(result)

{'encoding': 'iso8859_16', 'language': 'English', 'confidence': 1.0}


In [5]:

df = pd.read_csv('../data/solar-measurements_benin-malanville_qc.csv', encoding='latin1')

C:\Users\naomi.DESKTOP-T924HPN\AppData\Local\Temp\ipykernel_9688\685020699.py:1: DtypeWarning: Columns (1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('../data/solar-measurements_benin-malanville_qc.csv', encoding='latin1')


In [6]:
print(df.head())
print("\nColumns:", df.columns.tolist())

          Timestamp   GHI   DNI   DHI  ModA  ModB  Tamb    RH   WS WSgust  \
0  yyyy-mm-dd hh:mm  W/m²  W/m²  W/m²  W/m²  W/m²    °C     %  m/s    m/s   
1  2021-08-09 00:01  -1.2  -0.2  -1.1     0     0  26.2  93.4    0    0.4   
2  2021-08-09 00:02  -1.1  -0.2  -1.1     0     0  26.2  93.6    0      0   
3  2021-08-09 00:03  -1.1  -0.2  -1.1     0     0  26.2  93.7  0.3    1.1   
4  2021-08-09 00:04  -1.1  -0.1    -1     0     0  26.2  93.3  0.2    0.7   

  WSstdev            WD WDstdev   BP Cleaning Precipitation TModA TModB  \
0     m/s  °N (to east)       °  hPa   1 or 0        mm/min    °C    °C   
1     0.1         122.1       0  998        0             0  26.3  26.2   
2       0             0       0  998        0             0  26.3  26.2   
3     0.5         124.6     1.5  997        0             0  26.4  26.2   
4     0.4         120.3     1.3  997        0             0  26.4  26.3   

   Comments  
0       NaN  
1       NaN  
2       NaN  
3       NaN  
4       NaN  

C

In [15]:

print("Data Types:\n", df.dtypes)
print("\nSample Data:\n", df[['GHI', 'DNI', 'DHI', 'ModA', 'ModB', 'WS', 'WSgust']].head())

Data Types:
 Timestamp        object
GHI              object
DNI              object
DHI              object
ModA             object
ModB             object
Tamb             object
RH               object
WS               object
WSgust           object
WSstdev          object
WD               object
WDstdev          object
BP               object
Cleaning         object
Precipitation    object
TModA            object
TModB            object
dtype: object

Sample Data:
     GHI   DNI   DHI  ModA  ModB   WS WSgust
0  W/m²  W/m²  W/m²  W/m²  W/m²  m/s    m/s
1  -1.2  -0.2  -1.1     0     0    0    0.4
2  -1.1  -0.2  -1.1     0     0    0      0
3  -1.1  -0.2  -1.1     0     0  0.3    1.1
4  -1.1  -0.1    -1     0     0  0.2    0.7


In [7]:

summary_stats = df.describe()
print(summary_stats)

       Comments
count       0.0
mean        NaN
std         NaN
min         NaN
25%         NaN
50%         NaN
75%         NaN
max         NaN


In [8]:
# Missing values
missing_values = df.isna().sum()
missing_percent = (missing_values / len(df)) * 100
print("Missing Values:\n", missing_values)
print("\nColumns with >5% missing values:\n", missing_percent[missing_percent > 5])

Missing Values:
 Timestamp             0
GHI                   0
DNI                   0
DHI                   0
ModA                  0
ModB                  0
Tamb                  0
RH                    0
WS                    0
WSgust                0
WSstdev               0
WD                    0
WDstdev               0
BP                    0
Cleaning              0
Precipitation         0
TModA                 0
TModB                 0
Comments         525601
dtype: int64

Columns with >5% missing values:
 Comments    100.0
dtype: float64


In [9]:
# Drop columns with >50% missing values
df = df.loc[:, df.isna().mean() < 0.5]

In [16]:
key_columns = ['GHI', 'DNI', 'DHI', 'ModA', 'ModB', 'WS', 'WSgust']
for col in key_columns:
    non_numeric = df[col][pd.to_numeric(df[col], errors='coerce').isna() & df[col].notna()]
    if not non_numeric.empty:
        print(f"Non-numeric values in {col}:\n", non_numeric.unique())

Non-numeric values in GHI:
 ['W/m²']
Non-numeric values in DNI:
 ['W/m²']
Non-numeric values in DHI:
 ['W/m²']
Non-numeric values in ModA:
 ['W/m²']
Non-numeric values in ModB:
 ['W/m²']
Non-numeric values in WS:
 ['m/s']
Non-numeric values in WSgust:
 ['m/s']


In [ ]:

z_scores = df[key_columns].apply(zscore, nan_policy='omit')
# Flag outliers (|Z| > 3)
outliers = (z_scores.abs() > 3).any(axis=1)
print(f"Number of rows with outliers: {outliers.sum()}")

TypeError: can only concatenate str (not "float") to str

In [ ]:
# Impute missing values with median
for col in key_columns:
    df[col].fillna(df[col].median(), inplace=True)